# SVM (LIBSVM RBF) on GPU2 Features

This notebook trains an SVM classifier (LIBSVM via scikit-learn `SVC`) with RBF kernel (`C=10`, `gamma='auto'`) on features extracted by the GPU2 autoencoder, then evaluates performance.

Objectives:
- Extract features using trained encoder (already saved to binary files)
- Train SVM classifier on learned features
- Evaluate end-to-end classification performance

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from pathlib import Path
import time
import pickle

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Config
output_folder = Path("./output")  # Adjust as needed
feature_size = 8 * 8 * 128  # 8192 latent features from GPU2 encoder
expected_min, expected_max = 0.60, 0.65

## 1. Load Features from Binary Files

Create `FeatureDataLoader` to read train/test features and labels stored as binary files in `output_folder`. Validates shapes and counts.

In [2]:
class FeatureDataLoader:
    """
    Loads training and test features/labels from binary files.
    Features are float32 flattened arrays; labels are uint8.
    """
    def __init__(self, folder: Path, feature_size: int = 8192):
        self.folder = Path(folder)
        self.feature_size = feature_size
        self.num_classes = 10
    
    def _load_bin(self, path: Path, dtype):
        if not path.exists():
            raise FileNotFoundError(f"Missing file: {path}")
        return np.fromfile(path, dtype=dtype)
    
    def load_train(self):
        X_path = self.folder / "gpu_train_features.bin"
        y_path = self.folder / "train_labels.bin"
        X = self._load_bin(X_path, np.float32)
        y = self._load_bin(y_path, np.uint8)
        n = X.size // self.feature_size
        X = X.reshape(n, self.feature_size)
        if y.shape[0] != n:
            raise ValueError(f"Label count {y.shape[0]} != feature samples {n}")
        return X, y
    
    def load_test(self):
        X_path = self.folder / "gpu_test_features.bin"
        y_path = self.folder / "test_labels.bin"
        X = self._load_bin(X_path, np.float32)
        y = self._load_bin(y_path, np.uint8)
        n = X.size // self.feature_size
        X = X.reshape(n, self.feature_size)
        if y.shape[0] != n:
            raise ValueError(f"Label count {y.shape[0]} != feature samples {n}")
        return X, y

# Load with timing
print("Loading features and labels ...")
load_start = time.time()
loader = FeatureDataLoader(output_folder, feature_size)
train_features, train_labels = loader.load_train()
test_features, test_labels = loader.load_test()
load_time = time.time() - load_start
print(f"✓ Loaded: train {train_features.shape} | test {test_features.shape}")
print(f"Feature loading time: {load_time:.2f} s")

Loading features and labels ...
✓ Loaded: train (50000, 8192) | test (10000, 8192)
Feature loading time: 0.51 s


## 2. Data Preprocessing and Normalization

Normalize features using `StandardScaler` (fit on train, apply to test). Store scaler for production use.

In [4]:
# Handle any potential NaNs (shouldn't occur, but for safety)
train_features = np.nan_to_num(train_features)
test_features = np.nan_to_num(test_features)

# Standardize
print("Fitting StandardScaler on train features ...")
scaler = StandardScaler(with_mean=True, with_std=True)
scale_start = time.time()
scaler.fit(train_features)
train_features_scaled = scaler.transform(train_features)
test_features_scaled = scaler.transform(test_features)
scale_time = time.time() - scale_start

# Persist scaler
# scaler_path = output_folder / "scaler.pkl"
# with open(scaler_path, "wb") as f:
#     pickle.dump(scaler, f)
# print(f"✓ Scaler saved: {scaler_path}")

print(f"Scaling time: {scale_time:.2f} s")

Fitting StandardScaler on train features ...
Scaling time: 5.69 s


## 3. Initialize and Train SVM Model (RBF)

Train `SVC(kernel='rbf', C=10, gamma='auto')` and measure training time.

In [ ]:
print("\n" + "="*60)
print("SVM (RBF) Training Phase")
print("="*60)

svm = SVC(kernel='rbf', C=10, gamma='auto', verbose=1)
train_start = time.time()
# svm.fit(train_features_scaled, train_labels)
svm.fit(train_features, train_labels)
train_time = time.time() - train_start

model_path = output_folder / "svm_rbf_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(svm, f)
print(f"✓ Trained model saved: {model_path}")
print(f"SVM training time: {train_time:.2f} s")


SVM (RBF) Training Phase
[LibSVM]..
*.*
optimization finished, #iter = 3053
obj = -46228.868803, rho = -0.736170
nSV = 5521, nBSV = 5438


## 4. Make Predictions on Test Set

Predict on scaled test features and time inference.

In [ ]:
print("\n" + "="*60)
print("SVM (RBF) Prediction Phase")
print("="*60)

infer_start = time.time()
y_pred = svm.predict(test_features_scaled)
infer_time = time.time() - infer_start
print(f"✓ Inference time: {infer_time:.2f} s")
print(f"Predictions shape: {y_pred.shape} | Unique classes: {np.unique(y_pred)}")

## 5. Evaluate Classification Performance

Compute accuracy, confusion matrix, classification report, and compare to expected baseline (60–65%).

In [ ]:
accuracy = accuracy_score(test_labels, y_pred)
cm = confusion_matrix(test_labels, y_pred)
report = classification_report(test_labels, y_pred, target_names=[f"Class {i}" for i in range(10)], digits=4)

in_range = expected_min <= accuracy <= expected_max
range_text = "✓ Within expected range" if in_range else "⚠ Outside expected range"

print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Expected: {expected_min*100:.0f}% - {expected_max*100:.0f}% {range_text}\n")
print("Classification Report:\n" + report)

# Save metrics
metrics_path = output_folder / "svm_rbf_metrics.pkl"
with open(metrics_path, "wb") as f:
    pickle.dump({
        "accuracy": accuracy,
        "cm": cm,
        "report": report,
        "load_time": load_time,
        "scale_time": scale_time,
        "train_time": train_time,
        "infer_time": infer_time
    }, f)
print(f"✓ Metrics saved: {metrics_path}")

## 6. Visualize Confusion Matrix and Metrics

Plot heatmap, accuracy vs expected, per-class accuracy bars, and class distributions.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Confusion Matrix Heatmap
ax1 = axes[0, 0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=range(10), yticklabels=range(10), cbar_kws={'label': 'Count'})
ax1.set_title('Confusion Matrix - SVM RBF (C=10, gamma=auto)')
ax1.set_xlabel('Predicted Label')
ax1.set_ylabel('True Label')

# 2. Accuracy Comparison
ax2 = axes[0, 1]
categories = ['Model Accuracy', 'Expected Min', 'Expected Max']
values = [accuracy*100, expected_min*100, expected_max*100]
colors = ['#2ecc71' if (expected_min*100 <= values[0] <= expected_max*100) else '#e74c3c', '#3498db', '#3498db']
bars = ax2.bar(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Model Accuracy vs Expected Range')
ax2.set_ylim([0, 100])
ax2.axhline(y=expected_min*100, color='blue', linestyle='--', alpha=0.5)
ax2.axhline(y=expected_max*100, color='blue', linestyle='--', alpha=0.5)
for bar, val in zip(bars, values):
    ax2.text(bar.get_x() + bar.get_width()/2., val, f'{val:.2f}%', ha='center', va='bottom', fontsize=10)
ax2.grid(axis='y', alpha=0.3)

# 3. Per-class Accuracy
ax3 = axes[1, 0]
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
bars = ax3.bar(range(10), per_class_accuracy*100, color='#3498db', alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.set_xlabel('Class')
ax3.set_ylabel('Accuracy (%)')
ax3.set_title('Per-Class Accuracy')
ax3.set_ylim([0, 105])
ax3.set_xticks(range(10))
ax3.grid(axis='y', alpha=0.3)
for i, (bar, acc) in enumerate(zip(bars, per_class_accuracy)):
    ax3.text(bar.get_x() + bar.get_width()/2., acc*100, f'{acc*100:.1f}%', ha='center', va='bottom', fontsize=9)

# 4. Prediction Distribution
ax4 = axes[1, 1]
pred_counts = np.bincount(y_pred, minlength=10)
true_counts = np.bincount(test_labels, minlength=10)
x = np.arange(10)
width = 0.35
ax4.bar(x - width/2, true_counts, width, label='True', alpha=0.8, color='#2ecc71', edgecolor='black')
ax4.bar(x + width/2, pred_counts, width, label='Predicted', alpha=0.8, color='#3498db', edgecolor='black')
ax4.set_xlabel('Class')
ax4.set_ylabel('Count')
ax4.set_title('True vs Predicted Class Distribution')
ax4.set_xticks(x)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plot_path = output_folder / "svm_rbf_evaluation.png"
plt.savefig(str(plot_path), dpi=150, bbox_inches='tight')
print(f"✓ Visualization saved: {plot_path}")
plt.show()

## 7. Per-Class Accuracy Analysis

Generate detailed per-class metrics and identify easiest/hardest classes. Analyze animal vs vehicle confusion patterns.

In [ ]:
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

per_class_total = cm.sum(axis=1)
per_class_correct = cm.diagonal()
per_class_accuracy = per_class_correct / per_class_total

results_table = pd.DataFrame({
    'Class': range(10),
    'Name': cifar10_classes,
    'Total Samples': per_class_total,
    'Correct': per_class_correct,
    'Incorrect': per_class_total - per_class_correct,
    'Accuracy (%)': per_class_accuracy * 100
}).sort_values('Accuracy (%)', ascending=False)

print(results_table.to_string(index=False))

easiest_idx = int(results_table.iloc[0]['Class'])
hardest_idx = int(results_table.iloc[-1]['Class'])
print("\nKey Findings:")
print(f"✓ Easiest class: {cifar10_classes[easiest_idx]} (Class {easiest_idx}) - {results_table.iloc[0]['Accuracy (%)']:.2f}%")
print(f"✗ Hardest class: {cifar10_classes[hardest_idx]} (Class {hardest_idx}) - {results_table.iloc[-1]['Accuracy (%)']:.2f}%")
print(f"Accuracy range: {per_class_accuracy.min()*100:.2f}% - {per_class_accuracy.max()*100:.2f}%")
print(f"Std deviation: {per_class_accuracy.std()*100:.2f}%")

# Confusion pattern analysis
animal_classes = [2, 3, 4, 5, 6, 7]  # bird, cat, deer, dog, frog, horse
vehicle_classes = [0, 1, 8, 9]        # airplane, automobile, ship, truck
animal_confusion = cm[np.ix_(animal_classes, animal_classes)]
vehicle_confusion = cm[np.ix_(vehicle_classes, vehicle_classes)]
animal_confusion_rate = (animal_confusion.sum() - animal_confusion.diagonal().sum()) / animal_confusion.sum()
vehicle_confusion_rate = (vehicle_confusion.sum() - vehicle_confusion.diagonal().sum()) / vehicle_confusion.sum()
print(f"Animal-to-animal confusion rate: {animal_confusion_rate*100:.2f}%")
print(f"Vehicle-to-vehicle confusion rate: {vehicle_confusion_rate*100:.2f}%")

## 8. Compare with Baseline Methods

Comparison table: random baseline, linear SVM on raw pixels, end-to-end CNN, and this two-stage pipeline.

Phần này để điền kết quả của các version khác

In [ ]:
baseline = pd.DataFrame([
    {"Method": "SVM on GPU2 features (RBF)", "Accuracy": accuracy*100, "Training Time (s)": train_time, "Inference Time (s)": infer_time, "Notes": "This work"},
    {"Method": "Random baseline", "Accuracy": 10.0, "Training Time (s)": None, "Inference Time (s)": None, "Notes": "Chance level (10 classes)"},
    {"Method": "Linear SVM on raw pixels", "Accuracy": 40.0, "Training Time (s)": None, "Inference Time (s)": None, "Notes": "No feature learning"},
    {"Method": "End-to-end CNN (ResNet-18)", "Accuracy": 78.0, "Training Time (s)": None, "Inference Time (s)": None, "Notes": "Typical benchmark"}
])
print(baseline.to_string(index=False))